# Phase 2: Interactive Briefing (Gemini Powered)

이 노트는 **Phase 2: 기획 검토 및 확정** 단계를 구현합니다.
Phase 1에서 생성된 Draft Plan을 바탕으로, 사용자의 피드백을 반영하여 기획안을 점진적으로 개선합니다.
두 개의 특화된 에이전트(`ConfirmAgent`, `UpdateAgent`)가 협력하여 피드백을 분석하고 기획안을 수정합니다.

## 1. Environment & API Key Setup

In [ ]:
import os
import json
import time
import sys
import google.generativeai as genai

# TODO: API Key를 설정하세요
GOOGLE_API_KEY = "" 
genai.configure(api_key=GOOGLE_API_KEY)

/Users/jhkim/miniforge3/envs/Agent_EDU/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Example Input Data (From Phase 1)
Phase 1에서 생성된 Draft Plan 예시입니다. 실제 실행 시에는 이 데이터를 시작점으로 사용합니다.

In [2]:
initial_draft_plan = {
  "project_meta": {
    "title": "머신러닝의 기초",
    "target_audience": "대학교 1학년",
    "goal": "머신러닝 핵심 원리 습득 및 딥러닝 심화 학습을 위한 수학적·이론적 기반 구축"
  },
  "style_guide": {
    "tone": "객관적이고 엄밀한 학술 논문 스타일의 문체",
    "detail_level": "대학교 1학년 수준 (수학적 전개는 전문가 수준)",
    "math_policy": "전문가 수준의 엄밀한 수식 증명 및 전개 포함",
    "example_policy": "수식을 활용한 직접적인 계산 예시 및 이론적 사례 중심",
    "formatting": "학술 논문 표준 규격 (Abstract, Introduction, Methodology, Conclusion) 및 인용 체계 적용"
  },
  "chapters": [
    {
      "id": 1,
      "title": "머신러닝 개요 및 수학적 기초",
      "objective": "머신러닝의 정의와 선형대수학, 미분 등 기초 수학 지식을 복습한다.",
      "key_topics": [
        "지도학습 vs 비지도학습",
        "벡터와 행렬 연산",
        "편미분과 연쇄 법칙"
      ],
      "must_include": [
        "손실 함수의 수학적 정의"
      ]
    },
    {
      "id": 2,
      "title": "선형 회귀 (Linear Regression) 분석",
      "objective": "최소제곱법을 통한 선형 회귀의 수학적 도출 과정을 이해한다.",
      "key_topics": [
        "가설 함수(Hypothesis)",
        "비용 함수(Cost Function)",
        "정규 방정식(Normal Equation)"
      ],
      "must_include": [
        "행렬 미분을 이용한 파라미터 최적화 증명"
      ]
    },
    {
      "id": 3,
      "title": "경사하강법 (Gradient Descent) 최적화",
      "objective": "함수의 최솟값을 찾기 위한 반복적 최적화 알고리즘을 학습한다.",
      "key_topics": [
        "학습률(Learning Rate)",
        "전역 최적점과 지역 최적점",
        "Batch vs SGD vs Mini-batch"
      ],
      "must_include": [
        "경사하강법의 수렴 조건 및 계산 예시"
      ]
    },
    {
      "id": 4,
      "title": "로지스틱 회귀와 분류 이론",
      "objective": "이진 분류를 위한 확률론적 모델의 수식을 이해한다.",
      "key_topics": [
        "시그모이드 함수(Sigmoid Function)",
        "결정 경계(Decision Boundary)",
        "오즈비(Odds Ratio)"
      ],
      "must_include": [
        "최대우도법(MLE)을 통한 로스 유도"
      ]
    },
    {
      "id": 5,
      "title": "교차 엔트로피와 정보 이론",
      "objective": "분류 문제에서 사용되는 손실 함수의 정보이론적 배경을 학습한다.",
      "key_topics": [
        "엔트로피(Entropy)",
        "KL 다이버전스",
        "Cross-Entropy Loss"
      ],
      "must_include": [
        "Softmax 함수의 미분 및 역전파 기초"
      ]
    },
    {
      "id": 6,
      "title": "모델 평가 지표와 검증",
      "objective": "학습된 모델의 성능을 정량적으로 평가하는 방법을 익힌다.",
      "key_topics": [
        "Confusion Matrix",
        "Precision, Recall, F1-score",
        "ROC-AUC 곡선"
      ],
      "must_include": [
        "각 지표간의 트레이드오프 수식 분석"
      ]
    },
    {
      "id": 7,
      "title": "과적합 방지와 규제화 (Regularization)",
      "objective": "모델의 일반화 성능을 높이기 위한 수학적 기법을 배운다.",
      "key_topics": [
        "Bias-Variance Tradeoff",
        "L1 규제 (Lasso)",
        "L2 규제 (Ridge)"
      ],
      "must_include": [
        "라그랑주 승수법을 이용한 규제항의 해석"
      ]
    },
    {
      "id": 8,
      "title": "인공 신경망의 기초 (Perceptron)",
      "objective": "딥러닝의 기본 단위인 퍼셉트론의 구조와 한계를 이해한다.",
      "key_topics": [
        "단층 퍼셉트론",
        "XOR 문제와 비선형성",
        "활성화 함수(Activation Functions)"
      ],
      "must_include": [
        "퍼셉트론 수렴 정리"
      ]
    },
    {
      "id": 9,
      "title": "오차 역전파 (Backpropagation) 알고리즘",
      "objective": "다층 신경망을 학습시키기 위한 핵심 알고리즘의 수식을 전개한다.",
      "key_topics": [
        "Computational Graph",
        "Chain Rule 전개",
        "Gradient Flow"
      ],
      "must_include": [
        "다층 퍼셉트론에서의 가중치 업데이트 수식 유도"
      ]
    },
    {
      "id": 10,
      "title": "딥러닝으로의 확장",
      "objective": "머신러닝 기초를 바탕으로 심화 딥러닝 연구 방향을 제시한다.",
      "key_topics": [
        "심층 신경망(DNN) 구조",
        "기울기 소실 문제(Vanishing Gradient)",
        "향후 학습 로드맵"
      ],
      "must_include": [
        "현대적 딥러닝 프레임워크와 기초 모델 비교"
      ]
    }
  ]
}

## 3. Schemas & System Prompts
Confirm Agent와 Update Agent를 위한 JSON Schema와 System Prompt를 정의합니다.

In [3]:
# --- Confirm Agent Schema ---
confirm_schema = {
  "type": "object",
  "properties": {
    "is_approval": {
      "type": "boolean",
      "description": "사용자의 입력이 추가 변경 없이 승인/확인되었으면 true, 수정 요청이나 피드백이 있으면 false로 설정합니다."
    },
    "reasoning": {
      "type": "string",
      "description": "승인 여부에 대한 간략한 이유를 설명합니다."
    },
    "feedback_analysis": {
      "type": "object",
      "properties": {
        "summary": {
          "type": "string",
          "description": "사용자 피드백에 대한 한 문장 요약."
        },
        "action_items": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "target_category": {
                "type": "string",
                "enum": ["project_meta", "style_guide", "chapters", "general"],
                "description": "피드백이 적용될 대상 카테고리."
              },
              "target_id": {
                "type": "integer",
                "nullable": True,
                "description": "카테고리가 'chapters'일 때만 해당 챕터 ID를 입력하고, 나머지는 null로 설정합니다."
              },
              "target_field": {
                "type": "string",
                "nullable": True,
                "description": "수정할 구체적인 필드 이름. 예: title, goal, tone 등."
              },
              "intent": {
                "type": "string",
                "enum": ["modify", "create_new", "add_content", "delete", "reorder"],
                "description": "수정 의도/유형."
              },
              "instruction": {
                "type": "string",
                "description": "무엇을 어떻게 변경해야 하는지에 대한 자연어 지시사항."
              }
            },
            "required": ["target_category", "intent", "instruction"]
          }
        }
      },
      "required": ["summary", "action_items"]
    }
  },
  "required": ["is_approval", "feedback_analysis"]
}

CONFIRM_SYSTEM_PROMPT = """
You are the **Confirm Agent** for a Lecture Material Generation System.
Your goal is to analyze the user's feedback on a `Draft Plan` and determine if it is a confirmation (approval) or a request for modification.

### **Input Context**
1. **Current Draft Plan**: The full JSON object representing the current lecture plan (including existing chapters, IDs, and titles).
2. **User Prompt**: The user's natural language feedback regarding the `Current Draft Plan`.

### **Task Guidelines**
1. **Analyze Intent**:
   - If the user says "Perfect", "Go ahead", "Looks good", or "No changes needed" -> Set `"is_approval": true`.
   - If the user asks for changes, additions, deletions, or questions specific details -> Set `"is_approval": false`.

2. **Extract Action Items** (Only if `is_approval` is `false`):
   - Break down the user's request into discrete `action_items`.
   - Map each item to the specific part of the `Draft Plan` it affects.

3. **Enforce Schema Constraints (STRICT)**:
   - **`target_category`**: Must be one of `["project_meta", "style_guide", "chapters", "general"]`.
   - **`target_field`**: Must be a valid key existing in the `Draft Plan` under the selected category.
   - **`target_id`**: 
     - IF `target_category` is `"chapters"` AND intent is `modify/delete/reorder`: Provide the integer `id` of the target chapter.
     - IF intent is `create_new`: Set `target_id: null`. The `instruction` must specify *where* to add the chapter (e.g., "after Chapter 2").
     - For all other categories, set `target_id: null`.
"""

# --- Update Agent Schema ---
update_schema = {
  "type": "object",
  "properties": {
    "draft_plan": {
      "type": "object",
      "properties": {
        "project_meta": {
          "type": "object",
          "properties": {
            "title": { "type": "string" },
            "target_audience": { "type": "string" },
            "goal": { "type": "string" }
          },
          "required": ["title", "target_audience", "goal"]
        },
        "style_guide": {
          "type": "object",
          "properties": {
            "tone": { "type": "string" },
            "detail_level": { "type": "string" },
            "math_policy": { "type": "string" },
            "example_policy": { "type": "string" },
            "formatting": { "type": "string" }
          },
          "required": ["tone", "detail_level", "math_policy", "example_policy", "formatting"]
        },
        "chapters": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "id": { "type": "integer" },
              "title": { "type": "string" },
              "objective": { "type": "string" },
              "key_topics": { "type": "array", "items": { "type": "string" } },
              "must_include": { "type": "array", "items": { "type": "string" } }
            },
            "required": ["id", "title", "objective", "key_topics", "must_include"]
          }
        }
      },
      "required": ["project_meta", "style_guide", "chapters"]
    }
  },
  "required": ["draft_plan"]
}

UPDATE_SYSTEM_PROMPT = """
You are the **Update Agent** for a Lecture Material Generation System.
Your goal is to modify the `Current Draft Plan` based on the analysis of user feedback provided in `Feedback Analysis`.

### **Input Context**
1. **Current Draft Plan**: The JSON object representing the current state of the plan.
2. **Feedback Analysis**: A JSON object containing `action_items` derived from user feedback (output of Confirm Agent).

### **Execution Logic**
1. **Parse Action Items**:
   - Iterate through the `action_items` list in the `Feedback Analysis`.
   - Identify the `target_category` (project_meta, style_guide, chapters), `target_field`, `target_id`, and `intent`.

2. **Apply Modifications**:
   - **`modify`**: Update the value of the specified field.
   - **`add_content`**: Append the new value to the list (e.g., `key_topics`).
   - **`delete`**: Remove the specified item (e.g., a chapter by ID).
   - **`create_new`**: Create a new chapter object with inferred details and insert it at the appropriate position.
   - **`reorder`**: Change the order of chapters as requested.

3. **Constraints**:
   - **Preserve Unchanged Data**: Do NOT modify any fields that are not targeted by the action items.
   - **Maintain Schema**: Ensure the output JSON is a valid `Draft Plan` object inside the wrapper.
   - **Infer Missing Details**: When creating a new chapter, infer reasonable `objective`, `key_topics`, etc., if not explicitly provided, to keep the plan complete.
"""

## 4. Agent Implementation (ConfirmAgent & UpdateAgent)
각각의 역할을 수행하는 에이전트 클래스를 구현합니다.

In [4]:
class ConfirmAgent:
    def __init__(self, model_name="gemini-3-flash-preview"):
        self.model = genai.GenerativeModel(
            model_name=model_name,
            generation_config={
                "response_mime_type": "application/json",
                "response_schema": confirm_schema
            },
            system_instruction=CONFIRM_SYSTEM_PROMPT
        )

    def analyze(self, user_feedback, current_plan):
        prompt = f"""
        [Current Draft Plan]
        {json.dumps(current_plan, ensure_ascii=False)}

        [User Prompt]
        {user_feedback}
        """
        
        print("\n[Confirm Agent] Analyzing user feedback...")
        response = self.model.generate_content(prompt)
        return json.loads(response.text)

class UpdateAgent:
    def __init__(self, model_name="gemini-3-flash-preview"):
        self.model = genai.GenerativeModel(
            model_name=model_name,
            generation_config={
                "response_mime_type": "application/json",
                "response_schema": update_schema
            },
            system_instruction=UPDATE_SYSTEM_PROMPT
        )

    def modify_plan(self, current_plan, feedback_analysis):
        prompt = f"""
        [Current Draft Plan]
        {json.dumps(current_plan, ensure_ascii=False)}

        [Feedback Analysis]
        {json.dumps(feedback_analysis, ensure_ascii=False)}
        """
        
        print("\n[Update Agent] Modifying the plan based on feedback...")
        response = self.model.generate_content(prompt)
        return json.loads(response.text)["draft_plan"]

## 5. Phase 2 Execution Logic
사용자와의 상호작용 루프를 실행합니다.

In [5]:
def Execute_Phase2(draft_plan):
    # 1. 초기화
    confirm_agent = ConfirmAgent()
    update_agent = UpdateAgent()
    current_plan = draft_plan
    is_confirmed = False

    print("\n================ [Phase 2: Interactive Briefing] ================")

    # 2. 협의 및 수정 루프
    while not is_confirmed:
        # Step 1: 현재 계획 표시
        print("\n[Current Plan Summary]")
        print(f"Title: {current_plan.get('project_meta', {}).get('title', 'Untitled')}")
        print(f"Chapters: {len(current_plan.get('chapters', []))} predefined chapters")
        print("-"*30)
        print("기획안이 마음에 들면 '승인' 또는 '좋아'라고 입력하고, 수정을 원하면 구체적인 요청사항을 입력하세요.")
        
        sys.stdout.flush()
        time.sleep(0.5)

        # Step 2: 사용자 입력 수신
        user_feedback_prompt = input("\n피드백 입력 >>> ")

        # Step 3: 피드백 분석 (ConfirmAgent)
        try:
            analysis_result = confirm_agent.analyze(user_feedback_prompt, current_plan)
            
            # 결과 디버깅용 출력 (옵션)
            # print(f"[Analysis Result]: {analysis_result['is_approval']} / {analysis_result.get('reasoning')}")
            
            # Case A: 승인 (is_approval == True)
            if analysis_result.get("is_approval") is True:
                print("\n[System] 기획안이 확정되었습니다!")
                is_confirmed = True
                return current_plan

            # Case B: 수정 요청 (Update Plan)
            else:
                feedback_payload = analysis_result.get("feedback_analysis", {})
                print(f"\n[System] 수정 요청을 반영합니다: {feedback_payload.get('summary')}")
                
                # UpdateAgent가 계획을 수정함
                current_plan = update_agent.modify_plan(current_plan, feedback_payload)
                print("\n[System] 기획안이 업데이트되었습니다.")
                
        except Exception as e:
            print(f"\n[Error] 처리 중 오류가 발생했습니다: {e}")
            # 루프 계속 진행
            continue

## 6. Run Application
Phase 2를 단독으로 테스트합니다. (Phase 1의 결과물인 `initial_draft_plan`을 사용)

In [6]:
if __name__ == "__main__":
    if not GOOGLE_API_KEY:
        print("오류: 2번 셀에서 GOOGLE_API_KEY를 먼저 설정해주세요.")
    else:
        try:
            finalized_brief = Execute_Phase2(initial_draft_plan)
            
            print("\n" + "="*50)
            print(" [Finalized Brief Output] ")
            print("="*50)
            print(json.dumps(finalized_brief, indent=2, ensure_ascii=False))
            print("="*50)
        except KeyboardInterrupt:
            print("\n[System] 사용자에 의해 중단되었습니다.")


================ [Phase 2: Interactive Briefing] ================

[Current Plan Summary]
Title: 머신러닝의 기초
Chapters: 10 predefined chapters
------------------------------
기획안이 마음에 들면 '승인' 또는 '좋아'라고 입력하고, 수정을 원하면 구체적인 요청사항을 입력하세요.

[Confirm Agent] Analyzing user feedback...

[System] 수정 요청을 반영합니다: 대상 독자를 대학교 1학년에서 타 전공 출신 대학원 석사 수준으로 변경 요청

[Update Agent] Modifying the plan based on feedback...

[System] 기획안이 업데이트되었습니다.

[Current Plan Summary]
Title: 머신러닝의 기초
Chapters: 10 predefined chapters
------------------------------
기획안이 마음에 들면 '승인' 또는 '좋아'라고 입력하고, 수정을 원하면 구체적인 요청사항을 입력하세요.

[Confirm Agent] Analyzing user feedback...

[System] 수정 요청을 반영합니다: 사용자가 마지막 챕터인 '딥러닝으로의 확장'을 삭제해달라고 요청했습니다.

[Update Agent] Modifying the plan based on feedback...

[System] 기획안이 업데이트되었습니다.

[Current Plan Summary]
Title: 머신러닝의 기초
Chapters: 9 predefined chapters
------------------------------
기획안이 마음에 들면 '승인' 또는 '좋아'라고 입력하고, 수정을 원하면 구체적인 요청사항을 입력하세요.

[Confirm Agent] Analyzing user feedback...

[System] 기획안이 확정

================ [Phase 2: Interactive Briefing] ================

[Current Plan Summary]
Title: 머신러닝의 기초
Chapters: 10 predefined chapters
------------------------------
기획안이 마음에 들면 '승인' 또는 '좋아'라고 입력하고, 수정을 원하면 구체적인 요청사항을 입력하세요.

[Confirm Agent] Analyzing user feedback...

[System] 수정 요청을 반영합니다: 대상 독자를 대학교 1학년에서 타 전공 출신 대학원 석사 수준으로 변경 요청

[Update Agent] Modifying the plan based on feedback...

[System] 기획안이 업데이트되었습니다.

[Current Plan Summary]
Title: 머신러닝의 기초
Chapters: 10 predefined chapters
------------------------------
기획안이 마음에 들면 '승인' 또는 '좋아'라고 입력하고, 수정을 원하면 구체적인 요청사항을 입력하세요.

[Confirm Agent] Analyzing user feedback...

[System] 수정 요청을 반영합니다: 사용자가 마지막 챕터인 '딥러닝으로의 확장'을 삭제해달라고 요청했습니다.

[Update Agent] Modifying the plan based on feedback...

[System] 기획안이 업데이트되었습니다.

[Current Plan Summary]
Title: 머신러닝의 기초
Chapters: 9 predefined chapters
------------------------------
기획안이 마음에 들면 '승인' 또는 '좋아'라고 입력하고, 수정을 원하면 구체적인 요청사항을 입력하세요.

[Confirm Agent] Analyzing user feedback...

[System] 기획안이 확정되었습니다!

==================================================
 [Finalized Brief Output] 
==================================================
{
  "project_meta": {
    "title": "머신러닝의 기초",
    "target_audience": "타 전공 출신 대학원 석사 수준",
    "goal": "머신러닝 핵심 원리 습득 및 딥러닝 심화 학습을 위한 수학적·이론적 기반 구축"
  },
  "style_guide": {
    "tone": "객관적이고 엄밀한 학술 논문 스타일의 문체",
    "detail_level": "타 전공 출신 대학원생 수준 (수학적 전개는 전문가 수준)",
    "math_policy": "전문가 수준의 엄밀한 수식 증명 및 전개 포함",
    "example_policy": "수식을 활용한 직접적인 계산 예시 및 이론적 사례 중심",
    "formatting": "학술 논문 표준 규격 (Abstract, Introduction, Methodology, Conclusion) 및 인용 체계 적용"
  },
  "chapters": [
    {
      "id": 1,
      "title": "머신러닝 개요 및 수학적 기초",
      "objective": "머신러닝의 정의와 선형대수학, 미분 등 기초 수학 지식을 복습한다.",
      "key_topics": [
        "지도학습 vs 비지도학습",
        "벡터와 행렬 연산",
        "편미분과 연쇄 법칙"
      ],
      "must_include": [
        "손실 함수의 수학적 정의"
      ]
    },
    {
      "id": 2,
      "title": "선형 회귀 (Linear Regression) 분석",
      "objective": "최소제곱법을 통한 선형 회귀의 수학적 도출 과정을 이해한다.",
      "key_topics": [
        "가설 함수(Hypothesis)",
        "비용 함수(Cost Function)",
        "정규 방정식(Normal Equation)"
      ],
      "must_include": [
        "행렬 미분을 이용한 파라미터 최적화 증명"
      ]
    },
    {
      "id": 3,
      "title": "경사하강법 (Gradient Descent) 최적화",
      "objective": "함수의 최솟값을 찾기 위한 반복적 최적화 알고리즘을 학습한다.",
      "key_topics": [
        "학습률(Learning Rate)",
        "전역 최적점과 지역 최적점",
        "Batch vs SGD vs Mini-batch"
      ],
      "must_include": [
        "경사하강법의 수렴 조건 및 계산 예시"
      ]
    },
    {
      "id": 4,
      "title": "로지스틱 회귀와 분류 이론",
      "objective": "이진 분류를 위한 확률론적 모델의 수식을 이해한다.",
      "key_topics": [
        "시그모이드 함수(Sigmoid Function)",
        "결정 경계(Decision Boundary)",
        "오즈비(Odds Ratio)"
      ],
      "must_include": [
        "최대우도법(MLE)을 통한 로스 유도"
      ]
    },
    {
      "id": 5,
      "title": "교차 엔트로피와 정보 이론",
      "objective": "분류 문제에서 사용되는 손실 함수의 정보이론적 배경을 학습한다.",
      "key_topics": [
        "엔트로피(Entropy)",
        "KL 다이버전스",
        "Cross-Entropy Loss"
      ],
      "must_include": [
        "Softmax 함수의 미분 및 역전파 기초"
      ]
    },
    {
      "id": 6,
      "title": "모델 평가 지표와 검증",
      "objective": "학습된 모델의 성능을 정량적으로 평가하는 방법을 익힌다.",
      "key_topics": [
        "Confusion Matrix",
        "Precision, Recall, F1-score",
        "ROC-AUC 곡선"
      ],
      "must_include": [
        "각 지표간의 트레이드오프 수식 분석"
      ]
    },
    {
      "id": 7,
      "title": "과적합 방지와 규제화 (Regularization)",
      "objective": "모델의 일반화 성능을 높이기 위한 수학적 기법을 배운다.",
      "key_topics": [
        "Bias-Variance Tradeoff",
        "L1 규제 (Lasso)",
        "L2 규제 (Ridge)"
      ],
      "must_include": [
        "라그랑주 승수법을 이용한 규제항의 해석"
      ]
    },
    {
      "id": 8,
      "title": "인공 신경망의 기초 (Perceptron)",
      "objective": "딥러닝의 기본 단위인 퍼셉트론의 구조와 한계를 이해한다.",
      "key_topics": [
        "단층 퍼셉트론",
        "XOR 문제와 비선형성",
        "활성화 함수(Activation Functions)"
      ],
      "must_include": [
        "퍼셉트론 수렴 정리"
      ]
    },
    {
      "id": 9,
      "title": "오차 역전파 (Backpropagation) 알고리즘",
      "objective": "다층 신경망을 학습시키기 위한 핵심 알고리즘의 수식을 전개한다.",
      "key_topics": [
        "Computational Graph",
        "Chain Rule 전개",
        "Gradient Flow"
      ],
      "must_include": [
        "다층 퍼셉트론에서의 가중치 업데이트 수식 유도"
      ]
    }
  ]
}
==================================================